In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pymysql
from random import randint
import time

In [2]:
USER_AGENTS = [
 "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
 "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
 "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
 "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
 "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
 "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
 "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
 "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
 "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

In [3]:
def parsecontent(newsurl):
#     处理空url
    if newsurl == 'http://www.cuc.edu.cn':
        url = "http://www.cuc.edu.cn/news/2021/0309/c1901a178590/page.htm"
    else:
        url = newsurl
    
    random_agent = USER_AGENTS[randint(0, len(USER_AGENTS)-1)]
    headers = {'User-Agent':random_agent,}
    r = requests.get(url,headers=headers,verify=False)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rh,"html.parser")
    info = soup.find_all("span",attrs={"class":"arti-name"})
#     print(info)
    try:
        infotext = info[0].get_text()
        department = infotext[infotext.find('来源：')+3:infotext.find('20')]
        rn = re.compile('[\u4e00-\u9fa5]+')
        dep = rn.findall(department)
        date = infotext[infotext.find('20'):infotext.find('20')+10]
        countinfo = soup.find_all("span",attrs={"class":"WP_VisitCount"})
        count = countinfo[0].get_text()
        contentinfo = soup.find_all("div",attrs={"class":"wp_articlecontent"})
        content = contentinfo[0].get_text()
        return dep[0],date,count,content
    except:
        print("lose")
        return 0,0,0,0

In [4]:
def saveinfo(title,newsurl,department,date,count,content):
    db = pymysql.connect(host="localhost",user="zouzh",passwd="zouzihan0706",db="class")
    cursor = db.cursor()
    try:
        cursor.execute("INSERT INTO cucnews(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
        db.commit()
        i = i+1
#         print("db-ok")
    except:
#         print("db-error")
        db.rollback()
    db.close()

In [5]:
urlbegin = "http://www.cuc.edu.cn/news/1901/list"
# k用于计数
k = 0
for i in range(1,11):
    random_agent = USER_AGENTS[randint(0, len(USER_AGENTS)-1)]
    headers = {'User-Agent':random_agent,}
    url = urlbegin+str(i)+".htm"
    r = requests.get(url,headers=headers,verify=False)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rh,"html.parser")
    tit = soup.find_all("h3",attrs={'class','tit'})
    for t in tit:
        newsurl = t.find_all('a')
        urllen = str(newsurl[0]).find("target")
        title = t.get_text()
        newsurl = 'http://www.cuc.edu.cn' + (str(newsurl[0])[9:urllen-2])
#         对每个页面内容进行爬取
        department,date,count,content = parsecontent(newsurl)
#         测试
#         m = parsecontent(newsurl)
#         print(department)

        saveinfo(title,newsurl,department,date,count,content)
        k = k+1
        print(k)
#         print(title)
        time.sleep(0.5)
#         print(newsurl)
#         print(department)
#         print(date)
#         print(count)
#         print(content)
print("successful!")

[<span class="arti-name">
                            来源：保卫处
                              
                            2021-03-15
                              
                            <span class="info-name">作者：<span class="info-title"></span></span> 						
                            浏览量：<span id="hits"><span class="WP_VisitCount" url="/_visitcountdisplay?siteId=60&amp;type=3&amp;articleId=178723">40</span></span>
</span>]
1
[<span class="arti-name">
                            来源：教务处
                              
                            2021-03-15
                              
                            <span class="info-name">作者：<span class="info-title"></span></span> 						
                            浏览量：<span id="hits"><span class="WP_VisitCount" url="/_visitcountdisplay?siteId=60&amp;type=3&amp;articleId=178697">25</span></span>
</span>]
2
[<span class="arti-name">
                            来源：党委巡察工作办公室
                              
                            20